In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Solar Power Generation
Exploration of this dataset with several weird data values, and other oddities, that lead to finding some issues with the hardware of a plant. 
Further digging leads us to a conclusion: plant X has systemic problems. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt   
import seaborn as sns   # more plots
sns.set()

from dateutil.relativedelta import relativedelta #  dates with style
from scipy.optimize import minimize    # for function minimization

import statsmodels.formula.api as smf  # statistics and 
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

from itertools import product# some useful functions

import warnings    # `do not disturbe` mode
warnings.filterwarnings('ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
import plotly.express as px
import plotly.graph_objects as go
sns.set_context('notebook')
pd.options.display.max_rows = 102

In [ ]:
solar1GEN = pd.read_csv(r'/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv', parse_dates=['DATE_TIME'], infer_datetime_format=True )
#index_col=['Time'], 
solar1GEN.sample(12)

In [ ]:
solar1Sensor  = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv', parse_dates=['DATE_TIME'], infer_datetime_format=True )
solar2Sensor = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv', parse_dates=['DATE_TIME'], infer_datetime_format=True )
solar2GEN = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_2_Generation_Data.csv', parse_dates=['DATE_TIME'], infer_datetime_format=True )
solar2GEN["TOTAL_YIELDkWh"] =solar2GEN.TOTAL_YIELD*0.0036 # to kWh
solar2GEN.head(6)

In [ ]:
print("Mean yield in kWh plant 2:",round(solar2GEN["TOTAL_YIELDkWh"].mean()),"   Mean yield in kWh plant 1:", round(solar1GEN["TOTAL_YIELD"].mean()))

In [ ]:
solar1GEN['DATE'] = solar1GEN['DATE_TIME'].apply(lambda x:x.date())
solar2GEN['DATE'] = solar2GEN['DATE_TIME'].apply(lambda x:x.date())
solar1GEN['TIME'] = solar1GEN['DATE_TIME'].apply(lambda x:x.time())
solar2GEN['TIME'] = solar2GEN['DATE_TIME'].apply(lambda x:x.time())

We drop the PLANT_ID's to spare memory, and rename the inverter labels.

In [ ]:
solar1GEN= solar1GEN.drop("PLANT_ID", axis=1); 
solar2GEN= solar2GEN.drop("PLANT_ID", axis=1)

solar1GENdt= solar1GEN.set_index("DATE_TIME",  ) #inplace=True
solar2GENdt= solar2GEN.set_index("DATE_TIME",  )

solar1Sensor= solar1Sensor.drop("PLANT_ID", axis=1); 
solar2Sensor= solar2Sensor.drop("PLANT_ID", axis=1); 

solar1Sensordt= solar1Sensor.set_index("DATE_TIME",  )
solar2Sensordt= solar2Sensor.set_index("DATE_TIME",  )

solar1GEN= solar1GEN.sort_values(["SOURCE_KEY", "DATE_TIME"]) #sort_index()

First we rename these strange labels

In [ ]:
solar1GEN= solar1GEN.replace({'1BY6WEcLGh8j5v7':"A1", '1IF53ai7Xc0U56Y':"A2", '3PZuoBAID5Wc2HD':"A3",
   '7JYdWkrLSPkdwr4':"A4", 'McdE0feGgRqW7Ca':"A5", 'VHMLBKoKgIrUVDU':"A6",
   'WRmjgnKYAwPKWDb':"A10", 'ZnxXDlPa8U1GXgE':"A11", 'ZoEaEvLYb1n2sOq':"A12",
   'adLQvlD726eNBSB':"A13", 'bvBOhCH3iADSZry':"A14", 'iCRJl6heRkivqQ3':"A15",
   'ih0vzX44oOqAx2f':"A16", 'pkci93gMrogZuBj':"A17", 'rGa61gmuvPhdLxV':"A18",
   'sjndEbLyjtCKgGv':"A19", 'uHbuxQJl8lW7ozc':"A20", 'wCURE6d3bPkepu2':"A21",
   'z9Y9gH1T5YWrNuG':"A22", 'zBIq5rxdHJRwDNY':"A23", 'zVJPv84UY57bAof':"A24", 'YxYtjZvoooNbGkE':"A25"})

In [ ]:
code = {'1BY6WEcLGh8j5v7':"A1", '1IF53ai7Xc0U56Y':"A2", '3PZuoBAID5Wc2HD':"A3",'7JYdWkrLSPkdwr4':"A4", 'McdE0feGgRqW7Ca':"A5", 'VHMLBKoKgIrUVDU':"A6",
   'WRmjgnKYAwPKWDb':"A10", 'ZnxXDlPa8U1GXgE':"A11", 'ZoEaEvLYb1n2sOq':"A12", 'adLQvlD726eNBSB':"A13", 'bvBOhCH3iADSZry':"A14", 'iCRJl6heRkivqQ3':"A15",'ih0vzX44oOqAx2f':"A16", 'pkci93gMrogZuBj':"A17", 'rGa61gmuvPhdLxV':"A18",
   'sjndEbLyjtCKgGv':"A19", 'uHbuxQJl8lW7ozc':"A20", 'wCURE6d3bPkepu2':"A21", 'z9Y9gH1T5YWrNuG':"A22", 'zBIq5rxdHJRwDNY':"A23", 'zVJPv84UY57bAof':"A24", 'YxYtjZvoooNbGkE':"A25"}
label_df = pd.DataFrame.from_dict(code, orient='index', columns=['New_Code'])

In [ ]:
solar2GEN= solar2GEN.replace({'4UPUqMRk7TRMgml':"B1", '81aHJ1q11NBPMrL':"B2", '9kRcWv60rDACzjR':"B3",'Et9kgGMDl729KT4':"B4", 'IQ2d7wF4YD8zU1Q':"B5", 'LYwnQax7tkwH5Cb':"B6",
       'LlT2YUhhzqhg5Sw':"B7", 'Mx2yZCDsyf6DPfv':"B8", 'NgDl19wMapZy17u':"B9",'PeE6FRyGXUgsRhN':"B10", 'Qf4GUc1pJu5T6c6':"B11", 'Quc1TzYxW2pYoWX':"B12",
       'V94E5Ben1TlhnDV':"B13", 'WcxssY2VbP4hApt':"B14", 'mqwcsP2rE7J0TFp':"B15",'oZ35aAeoifZaQzV':"B16", 'oZZkBaNadn6DNKz':"B17", 'q49J1IKaHRwDQnt':"B18",
       'rrq4fwE8jgrTyWY':"B19", 'vOuJvMaM2sgwLmb':"B20", 'xMbIugepa2P7lBB':"B21",'xoJJ8DcxJEcupym':"B22"})

In [ ]:
sources1 =solar1GEN.SOURCE_KEY.unique(); sources2 =solar2GEN.SOURCE_KEY.unique()

In [ ]:
def roundingP1(df):
    df['DC_POWER'] = round(df['DC_POWER'],2)
    df['AC_POWER'] = round(df['AC_POWER'],2)
    df['DAILY_YIELD'] = round(df['DAILY_YIELD'],2)
    df['TOTAL_YIELD'] = round(df['TOTAL_YIELD'],2)

Limit temperature and irradiation values' level of precision

In [ ]:
def roundingP2(df):
    df['AMBIENT_TEMP'] = round(df['AMBIENT_TEMP'],2)
    df['MODULE_TEMP'] = round(df['MODULE_TEMP'],2)
    df['IRRADIATION'] = round(df['IRRADIATION'],4)

#### Ideas for adding features are : 
- efficiency of Inverter= AC_POWER/DC_POWER  
- DAILY_YIELD increments 
- inverter age i.f.o. max/relative TOTAL_YIELD  

In [ ]:
gen1_dailymax=solar1GENdt.resample("D").max()  # better when 1 selected
gen1_dailymax.head(15)

In [ ]:
gen2_dailymax=solar2GENdt.resample("D").max()  # better when 1 selected
gen2_dailymax.head(5)

### Rearrange data of each Inverter 
In the meantime we perform a resampling to daily values. 

In [ ]:
def Inverterextract(df):
    global plant1
    plant1= pd.DataFrame(index=gen1_dailymax.index, data=gen1_dailymax.iloc[:,0])  #
    
    for b in sources1:
        dfs= df[df.SOURCE_KEY== b] #
        dfsi=dfs.set_index("DATE_TIME")

        invert =dfsi.resample("D").agg({'AC_POWER':np.sum, "DAILY_YIELD":np.max}) #'DC_POWER': np.sum, "DAILY_YIELD":np.max,"TOTAL_YIELD":"max"  lambda x: np.std(x, ddof=1)
        invert.columns = invert.columns+'_'+str(b)
        plant1= pd.concat([plant1, invert], axis=1)
        next   #

In [ ]:
def Inverterextract_2(df):
    global plant2
    plant2= pd.DataFrame( index=gen1_dailymax.index, data=gen1_dailymax.iloc[:,0]) # [2]
    
    for b in sources2:
        dfs= df[df.SOURCE_KEY== b] #
        dfsi=dfs.set_index("DATE_TIME")

        invert =dfsi.resample("D").agg({'AC_POWER':np.sum, "DAILY_YIELD":np.max}) #'DC_POWER': np.sum,"TOTAL_YIELD":"max"  lambda x: np.std(x, ddof=1)
        invert.columns = invert.columns+'_'+str(b)
        plant2= pd.concat([plant2, invert], axis=1)#
        next #

In [ ]:
Inverterextract(solar1GEN)
#if isinstance(gen1_dailymax):
roundingP1(gen1_dailymax); gen1_dailymax.head()

In [ ]:
Inverterextract_2(solar2GEN)

In [ ]:
plant1.head(15)

In [ ]:
Invertors1=plant1[["AC_POWER_A1","DAILY_YIELD_A1","AC_POWER_A2","DAILY_YIELD_A2","AC_POWER_A3","DAILY_YIELD_A3","AC_POWER_A4","DAILY_YIELD_A4",
                   "AC_POWER_A5","AC_POWER_A6","DAILY_YIELD_A6",#"AC_POWER_A7",
                   "AC_POWER_A10","DAILY_YIELD_A10","AC_POWER_A11","DAILY_YIELD_A11","AC_POWER_A12","DAILY_YIELD_A12","AC_POWER_A13","DAILY_YIELD_A13",
                   "AC_POWER_A14","DAILY_YIELD_A14","AC_POWER_A15","AC_POWER_A16","DAILY_YIELD_A16","AC_POWER_A17","DAILY_YIELD_A17","AC_POWER_A18","DAILY_YIELD_A18",
                   "AC_POWER_A19","DAILY_YIELD_A19","AC_POWER_A20","DAILY_YIELD_A20","AC_POWER_A21","DAILY_YIELD_A21","AC_POWER_A22","DAILY_YIELD_A22",
                   "AC_POWER_A23","DAILY_YIELD_A23","AC_POWER_A24","DAILY_YIELD_A24","AC_POWER_A25","DAILY_YIELD_A25"]]

In [ ]:
filter_col = [col for col in Invertors1 if col.startswith('A')]
#filter_col
Invertors1_AC=Invertors1[filter_col]
Invertors1_AC.head()

### Plants 1 & 2 production compared
Plots of the generated and conversed energy (AC) vs. the daily cumulated energy (yield values). 

In [ ]:
plt.figure(figsize=(16, 6))
sns.lineplot(gen1_dailymax.index, data=plant1, y="AC_POWER_A1",label="AC_POWER_A1",)
sns.lineplot( gen1_dailymax.index, data=plant1, y="DAILY_YIELD_A1",label="DAILY_YIELD_A1", ); 
sns.lineplot(gen1_dailymax.index, data=plant2, y="AC_POWER_B1",label="AC_POWER_B1",)
sns.lineplot( gen1_dailymax.index, data=plant2, y="DAILY_YIELD_B1", label="DAILY_YIELD_B1",); 

In [ ]:
plt.figure(figsize=(16, 6))
sns.lineplot(gen1_dailymax.index, data=plant1, y="AC_POWER_A19",label="AC plant1 A19",)
sns.lineplot( gen1_dailymax.index, data=plant1, y="DAILY_YIELD_A19",label="DAILY YIELD A19", ); 
sns.lineplot(gen1_dailymax.index, data=plant2, y="AC_POWER_B19",label="AC plant2 B19",)
sns.lineplot( gen1_dailymax.index, data=plant2, y="DAILY_YIELD_B19", label="DAILY YIELD plant2 B19",);

- Plant 2 is less productive, perhaps due to smaller size or older panel age.
- Anyway, the ambient temperature at plant 2 is a little higher on average. This has an small negative effect on the solar panels efficiency.
    - Also, higher inner temperatures of the inverter, e.g. due to conversion, may cause the inverters to "derate" sooner than those of plant 1.
    - derate = to produce less energy thru conversion while being out of manufacturer-intended working range. *derate = to produce less energy thru conversion while being out of manufacturer-intended working range*. 

N.B.: in topic <a href="#B2">"Inverter B2"</a> one cause of reduced production will be revealed. 

### Plant 1 daily yields over 34 days
We can notice about 10 days with yield drops from top to bottom. 

In [ ]:
plt.figure(figsize=(16, 6))

sns.lineplot(x=solar1GEN.index, y="DAILY_YIELD", data=solar1GEN,color="forestgreen"); # kind="marker"
#plt.plot(ads.Ads, )
plt.title('E generated (15 min. data)')
plt.grid(True)
plt.show()

### Inverter A1

In [ ]:
solar1GEN_A1= solar1GEN[solar1GEN.SOURCE_KEY =="A1"]
solar1GEN_A1

In [ ]:
import datetime 
from dateutil.rrule import (rrule, MO, TU, WE, TH, FR, SA, SU, YEARLY, MONTHLY, WEEKLY, DAILY, HOURLY, MINUTELY, SECONDLY)
solar1GEN_A1.set_index("DATE_TIME", inplace=True)
plt.figure(figsize=(18, 5)); solar1GEN_A1.DAILY_YIELD.plot(); 

In [ ]:
solar1GEN_A10= solar1GEN[solar1GEN.SOURCE_KEY =="A10"]
#solar1GEN_A10.sample(10)
solar1GEN_A10.set_index("DATE_TIME", inplace=True)
plt.figure(figsize=(18, 5)); solar1GEN_A1.DAILY_YIELD.plot(); 
solar1GEN_A10.DAILY_YIELD.plot( c='indigo'); 

## Inverter age estimation and comparison
This is an attempt to get an estimate of inverter ages. This will turn out to be very difficult as the plant 2 total yield values are more than 1000 times that of plant 1.  

In [ ]:
print(solar1GEN.TOTAL_YIELD.max(), solar2GEN.TOTAL_YIELD.max(),"Ratio:" ,round(solar1GEN.TOTAL_YIELD.max()/solar2GEN.TOTAL_YIELD.max(),3))

In [ ]:
print(solar1GEN.TOTAL_YIELD.max(), solar2GEN.TOTAL_YIELDkWh.max(),"Ratio:" ,round(solar1GEN.TOTAL_YIELD.max()/solar2GEN.TOTAL_YIELDkWh.max(),3))

In [ ]:
solar1GEN["Age"] =solar1GEN.TOTAL_YIELD/solar1GEN.TOTAL_YIELD.max()

solar2GEN["Age"] =solar2GEN.TOTAL_YIELD/solar2GEN.TOTAL_YIELD.max()

solar2GEN["Age_kWh"] =solar2GEN.TOTAL_YIELD/solar2GEN.TOTAL_YIELDkWh.max()

In [ ]:
sns.distplot(solar1GEN["Age"], bins=50); #  module 'seaborn' has no attribute 'displot'

In [ ]:
sns.distplot(solar2GEN["Age"], bins=50); 

these est. of inverter age are not credible, since pv panels are not so long commercially available. 

#### Average daily yield plant 1 and 2

In [ ]:
dta= pd.to_datetime("2020-06-17 09:30:00"); #dtb= pd.to_datetime("2020-05-23 19:00:00")
solar1GEN_17june= solar1GEN[solar1GEN.DATE_TIME== dta] #
solar1GEN_17june["Age"] =solar1GEN_17june.TOTAL_YIELD/solar1GEN_17june.TOTAL_YIELD.max()
sns.distplot(solar1GEN_17june["Age"], bins=50); 

In [ ]:
dta= pd.to_datetime("2020-06-17 05:30:00"); dtz= pd.to_datetime("2020-06-17 19:30:00"); #
solar2GEN_17june= solar2GEN[solar2GEN.DATE_TIME>= dta] ; solar2GEN_17june= solar2GEN_17june[solar2GEN_17june.DATE_TIME< dtz] 
# no 0 values
solar2GEN_17june= solar2GEN_17june[solar2GEN_17june.DAILY_YIELD !=0]
solar2GEN_17june= solar2GEN_17june[solar2GEN_17june.AC_POWER !=0]
billion= solar2GEN_17june#[solar2GEN_17june.TOTAL_YIELD>1E9] # 2080.43 TOTAL_YIELD>1E9
billion.DAILY_YIELD.mean()

is the underperformance real or perceived? 
Hardware or software cause? 

In [ ]:
dta= pd.to_datetime("2020-06-17 05:30:00"); dtz= pd.to_datetime("2020-06-17 19:30:00"); #
solar1GEN_17june= solar1GEN[solar1GEN.DATE_TIME>= dta] ; solar1GEN_17june= solar1GEN_17june[solar1GEN_17june.DATE_TIME< dtz] 
# no 0 values
solar1GEN= solar1GEN[solar1GEN.DAILY_YIELD !=0]
solar1GEN= solar1GEN[solar1GEN.AC_POWER !=0]
million= solar1GEN_17june#[solar1GEN_17june.TOTAL_YIELD>1E6] # 3452.55 TOTAL_YIELD>1E6
million.DAILY_YIELD.mean()

## The daily maximum yield per inverter

In [ ]:
lastyield2 =solar2GEN.groupby(["DATE","SOURCE_KEY"])[["DAILY_YIELD"]].last() # [["DAILY_YIELD"]]
lastyield2 = lastyield2.reset_index( level="SOURCE_KEY")
lastyield2.index = pd.to_datetime( lastyield2.index)
#lastyield2     average=  maximum= ,"SOURCE_KEY"

In [ ]:
lastyield1 =solar1GEN.groupby(["DATE","SOURCE_KEY"])[["DAILY_YIELD"]].last() # [["DAILY_YIELD"]]
lastyield1 = lastyield1.reset_index( level="SOURCE_KEY")
lastyield1.index = pd.to_datetime( lastyield1.index)

### plant comparison of daily yields

In [ ]:
#sns.set_theme()
cmap = sns.palplot(sns.diverging_palette(250.0/250, 145.0/250, s=200/250, l=9, n=22))

In [ ]:
fig, axe= plt.subplots(2,1, figsize=(20, 12)) #
sns.lineplot(x=lastyield1.index, y="DAILY_YIELD", data=lastyield1,  ax=axe[0],lw=0.7, hue="SOURCE_KEY"); #color=cmap,
sns.lineplot(x=lastyield2.index, y="DAILY_YIELD", data=lastyield2,  ax=axe[1],lw=0.7, hue="SOURCE_KEY");
#plt.title('Daily yields generated (15 min. data)')
plt.grid(True); plt.show()

In [ ]:
KWHyield2 =solar2GEN.groupby(["DATE","SOURCE_KEY"])[["TOTAL_YIELDkWh"]].last() # [["DAILY_YIELD"]]
KWHyield2 = KWHyield2.reset_index( level="SOURCE_KEY")

KWHyield2.index = pd.to_datetime( KWHyield2.index)

In [ ]:
KWHyield1 =solar1GEN.groupby(["DATE","SOURCE_KEY"])[["TOTAL_YIELD"]].last() # max
KWHyield1 = KWHyield1.reset_index( level="SOURCE_KEY")
KWHyield1.index = pd.to_datetime( KWHyield1.index)

### Total yield plant 1

In [ ]:
KWHyield1 =solar1GEN.groupby(["DATE_TIME","SOURCE_KEY"])[["TOTAL_YIELD"]].last() # max
KWHyield1 = KWHyield1.reset_index( level="SOURCE_KEY")
KWHyield1.index = pd.to_datetime( KWHyield1.index)

In [ ]:
fig, axe= plt.subplots(1,1, figsize=(12, 8)) #p
sns.lineplot(x=KWHyield1.index, y="TOTAL_YIELD", data=KWHyield1, color=cmap, lw=0.7, hue="SOURCE_KEY"); #ax=axe[0]

Generator 1 has step by step increases in yields.

In [ ]:
fig, axe= plt.subplots(1,1, figsize=(12, 8)) #p
sns.lineplot(x=KWHyield2.index, y="TOTAL_YIELDkWh", data=KWHyield2, color=cmap, lw=0.7, hue="SOURCE_KEY"); #ax=axe[0]

What are these interruptions? Mind they don't happen on the same day, so I exclude some deliberate shut down. 

### Total yield plant 2
A conversion of the total yield values of plant 2 had to be done in order to get values which we can compare with plant 1. 

In [ ]:
KWHyield2 =solar2GEN.groupby(["DATE_TIME","SOURCE_KEY"])[["TOTAL_YIELDkWh"]].last() # [["DAILY_YIELD"]]
KWHyield2 = KWHyield2.reset_index( level="SOURCE_KEY")
KWHyield2.index = pd.to_datetime( KWHyield2.index)
KWHyield2    # average=  maximum= ,"SOURCE_KEY"

In [ ]:
KWHyield2.to_csv("/kaggle/working/KWHyield2.csv")

### Total yields for all Plant 2 inverters
And what happens if we take feature "date_time" for a finer timing...
Blibs are appearing everywhere, and they range in size and duration.

In [ ]:
KWHyield2 =solar2GEN.groupby(["DATE_TIME","SOURCE_KEY"])[["TOTAL_YIELDkWh"]].max() # [["DAILY_YIELD"]]
KWHyield2 = KWHyield2.reset_index( level="SOURCE_KEY")
KWHyield2.index = pd.to_datetime( KWHyield2.index)

In [ ]:
fig, axe= plt.subplots(1,1, figsize=(20, 12)) #
plt.get_cmap('jet')
sns.lineplot(x=KWHyield2.index, y="TOTAL_YIELDkWh", data=KWHyield2, color=cmap, lw=0.7, hue="SOURCE_KEY"); #ax=axe[0]

Inverter B14 has some very odd behavior concerning total yield values:  
- 2020-05-29 17:30:00,B14,654535.0888799999  
- 2020-05-29 17:45:00,B14,610899.5784  
- 2020-05-29 18:00:00,B14,654535.3710857143  

Moreover, several other inverters of this plant deliver non consistent total yield values. See inverter B2 f.i.  
**I start to suspect this plant has some systemic issue, which was not disclosed by the donor of the dataset.**

In [ ]:
B2 =KWHyield2[KWHyield2.SOURCE_KEY =="B2"]; 
fig, axe= plt.subplots(1,1, figsize=(20, 4))
sns.lineplot(x=B2.index, y="TOTAL_YIELDkWh", data=B2); 

In [ ]:
B2.head()

In [ ]:
M24mayblib= B2[(B2.index>="2020-05-24 06:00:00") &(B2.index<"2020-05-24 23:00:00")]
M24may_2sensor = solar2Sensor[(solar2Sensor.DATE_TIME>="2020-05-24 06:00:00") &(solar2Sensor.DATE_TIME<"2020-05-24 23:00:00")]

### <a id="B2">Inverter B2</a>: inner temperature and read out of total yields
The total yields values are supposed to only increase in time, not to decrease. But during the heat of the day, this is indeed what happens to B2. 
In the plot the yield starts to drop above 52°C, and power generation stops soon after this, even before reaching 55°C.
On some panel data sheets you can find the I V-curves: current against voltage curve. The current stays high and constant, a horizontal line, but above say 30V, current will quickly drop to 0. Now as $P=U∗I$, resulting power will be 0.

In [ ]:
fig, axe= plt.subplots(1,1, figsize=(12, 5))
sns.lineplot( x=M24may_2sensor.index ,y=M24may_2sensor.MODULE_TEMPERATURE, data=M24may_2sensor)#"DATE_TIME"
sns.lineplot(  x=M24may_2sensor.index,y=M24mayblib.TOTAL_YIELDkWh/50000, data=M24mayblib); 
plt.grid(b=True,which='minor', axis="both"); plt.ylim=(25,90)
axe.set( ylim=(25,70));  

In [ ]:
fig, axe= plt.subplots(1,1, figsize=(12, 5))
sns.lineplot( x=M24may_2sensor.index,y=M24may_2sensor.MODULE_TEMPERATURE, data=M24may_2sensor)
sns.lineplot(  x=M24may_2sensor.index,y=M24mayblib.TOTAL_YIELDkWh/50000, data=M24mayblib); 
plt.grid(b=True,which='minor', axis="both"); 
axe.set( ylim=(5,90));  

### <a id="B3">Inverter B3</a>: inner temperature and read out of total yields

In [ ]:
B3 =KWHyield2[KWHyield2.SOURCE_KEY =="B3"]; 
fig, axe= plt.subplots(1,1, figsize=(18, 4))
sns.lineplot(  x=B3.index,y=B3.TOTAL_YIELDkWh, data=B3); 

In [ ]:
M24maybliB3= B3[(B3.index>="2020-06-02 06:00:00") &(B3.index<"2020-06-04 23:00:00")]
M24may_2sensorB3 = solar2Sensor[(solar2Sensor.DATE_TIME>="2020-06-02 06:00:00") &(solar2Sensor.DATE_TIME<"2020-06-04 23:00:00")]
fig, axe= plt.subplots(1,1, figsize=(12, 5))
sns.lineplot( x=M24may_2sensorB3.index,y=M24may_2sensorB3.MODULE_TEMPERATURE, data=M24may_2sensorB3)
sns.lineplot(  x=M24may_2sensorB3.index,y=M24maybliB3.TOTAL_YIELDkWh/50000, data=M24maybliB3); 
plt.grid(b=True,which='minor', axis="both"); plt.title("Inverter B3 ECG")
axe.set( ylim=(5,90)); 

The fluctuations have a negative impact on the average total  yield of plant 2.

In [ ]:
billion.DAILY_YIELD.mean()/billion.DAILY_YIELD.max() #0.4534437633686215

In [ ]:
million.DAILY_YIELD.mean()/million.DAILY_YIELD.max()#0.5236844125145991

### Filter out power drops below 50% daily yield.
therefore we need merged data:
#### Plant 2, 2020-05-23

In [ ]:
dta= pd.to_datetime("2020-05-23 05:30:00"); dtb= pd.to_datetime("2020-05-23 19:30:00")
Sensor2dt_mei23_= solar2Sensordt[solar2Sensordt.index>= dta] #"2020-5-23 05:30:00"
Sensor2dt_mei23_= Sensor2dt_mei23_[Sensor2dt_mei23_.index <=dtb]; #

In [ ]:
dta= pd.to_datetime("2020-05-01 05:30:00"); dtb= pd.to_datetime("2020-05-31 19:30:00")
Sensor2dt_mei= solar2Sensordt[solar2Sensordt.index>= dta] #"2020-5-23 05:30:00"
Sensor2dt_mei= Sensor2dt_mei[Sensor2dt_mei.index <=dtb]; #

In [ ]:
mei23B_merg = Sensor2dt_mei23_.merge(solar2GEN,left_index=True, right_on="DATE_TIME" )

In [ ]:
mei23B_merg= mei23B_merg.loc[(mei23B_merg.AC_POWER !=0)] # weed out 0
TOTAL_YIELDkWh_avg= np.mean(mei23B_merg.TOTAL_YIELDkWh); TOTAL_YIELDkWh_avg #

In [ ]:
mei23B_merg.sample()

In [ ]:
DAILY_YIELD_avg= np.mean(mei23B_merg.DAILY_YIELD); DAILY_YIELD_avg #DAILY_YIELD

In [ ]:
mei23B_merg= mei23B_merg.rename(columns={"SOURCE_KEY_y": "SOURCE_KEY"})

bagger= mei23B_merg.loc[(mei23B_merg.MODULE_TEMPERATURE >50)& ( mei23B_merg.DAILY_YIELD < DAILY_YIELD_avg)&(mei23B_merg.AC_POWER !=0)]
baggermean= bagger.groupby("SOURCE_KEY")["MODULE_TEMPERATURE"].agg( size= np.size)

baggermean.tail()

In [ ]:
bagger_max= bagger.groupby("SOURCE_KEY")["MODULE_TEMPERATURE"].agg( Min= min, mean= np.mean,Max= max, size= np.size)

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(12, 5) ); bagger_maxx=bagger_max.loc[:,["Min","mean","Max"]]

sns.scatterplot(  data=bagger_maxx, ax=ax);

B14 is again the odd duck as this inverter amount of yield values is much higher.
But perhaps it is caused by the fact that this inverter is the last man standing.

Now the groups deployed in the plant 2 formation have appeared:

    1,11,13,16,17,20,8
    10,22,4
    12,2, and perhaps 14

#### Plant 2, whole month May

In [ ]:
mei_B_merg = Sensor2dt_mei.merge(solar2GEN,left_index=True, right_on="DATE_TIME" )

#mei_B_merg = mei_B_merg.loc[(mei_B_merg.AC_POWER !=0)]
mei_B_merg.info()

In [ ]:
DAILY_YIELD_avg1= np.mean(mei_B_merg.DAILY_YIELD); DAILY_YIELD_avg1 #

We filter out records with panel temperatures above 50°, daily yields less than average AND ac current (power) isnot 0.

In [ ]:
mei_B_merg= mei_B_merg.rename(columns={"SOURCE_KEY_y": "SOURCE_KEY"})

bagB= mei_B_merg.loc[(mei_B_merg.MODULE_TEMPERATURE >50)& ( mei_B_merg.DAILY_YIELD < DAILY_YIELD_avg)& (mei_B_merg.AC_POWER !=0)]
bagB_mean= bagB.groupby("SOURCE_KEY")["MODULE_TEMPERATURE"].agg( Min= min, mean= np.mean, Max= max, size= np.size) #

bagB_mean.head()

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(15, 7) ); bagB_meanx=bagB_mean.loc[:,["Min","mean","Max"]]

sns.scatterplot(  data=bagB_meanx, ax=ax); #
ax2 = plt.twinx()
sns.scatterplot(x=bagB_mean.index, y="size",data=bagB_mean, color="red",marker="^", label="Size",ax=ax2); #"SOURCE_KEY"
ax2.figure.legend();

>**Above average alerts are coming from inverters B12,B19, B4, B7, B13 and perhaps B3.  
These are good candidates for inspection and replacement.**

### Filter out power drops below 50% daily yield: Plant 1
some normal behavior is here expected

In [ ]:
dta= pd.to_datetime("2020-05-23 05:30:00"); dtb= pd.to_datetime("2020-05-23 19:30:00")
Sensor1dt_mei23_= solar1Sensordt[solar1Sensordt.index>= dta] #"2020-5-23 05:30:00"
Sensor1dt_mei23_= Sensor1dt_mei23_[Sensor1dt_mei23_.index <=dtb]; #

In [ ]:
mei23A_merg = Sensor1dt_mei23_.merge(solar1GEN,left_index=True, right_on="DATE_TIME" )

In [ ]:
DAILY_YIELD_avg1= np.mean(mei23A_merg.DAILY_YIELD); DAILY_YIELD_avg1 #DAILY_YIELD

In [ ]:
bagA= mei23A_merg.loc[(mei23A_merg.MODULE_TEMPERATURE >50)& ( mei23A_merg.DAILY_YIELD < DAILY_YIELD_avg)& (mei23A_merg.AC_POWER !=0)]
bagA_mean= bagA.groupby("SOURCE_KEY_y")["MODULE_TEMPERATURE"].agg( Min= min, mean= np.mean, Max= max, ) # size= np.size

bagA_mean.head()

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(12, 5) ); #bagA_meanx=bagA_mean.loc[:,["Min","mean","Max"]]

sns.scatterplot(  data=bagA_mean, ax=ax); # hue="SOURCE_KEY"

In the heatmap the endurance of A1 and A14 was visible by their bright timetrack, and now inverter 18 may join them.

## Resampling generator data to daily values
We try to have a better view on what is happening by viewing daily data.  
There appears to be a 10-fold error in a series values: DC power for plant 1. 

In [ ]:
solar1GEN["DC_POWER"] = solar1GEN.DC_POWER/10
solar1GENdt["DC_POWER"]= solar1GENdt.DC_POWER/10
df1_DC = solar1GENdt.DC_POWER.resample('D').sum()
df1_AC = solar1GENdt.AC_POWER.resample('D').sum()
df1_DY = solar1GENdt.DAILY_YIELD.resample('D').last() #.max()
df1_TY = solar1GENdt.TOTAL_YIELD.resample('D').last() #.max()
df1_DC.tail()

>to be cont...

In [ ]:
df = pd.merge( df1_DC, df1_AC, left_index=True, right_index=True) 
df_ = pd.merge( df1_DY ,df1_TY, left_index=True, right_index=True) 
df_1 = pd.merge( df ,df_, left_index=True, right_index=True) 

df_1["Effic"]=df_1.AC_POWER /df_1.DC_POWER 
roundingP1(df_1)
df_1.tail(15)

In [ ]:
#df_1.info()

A conversion efficiency of 98% seems ok to me. 

### Resampling Generator 2 data

In [ ]:
df2_DC = solar2GENdt.DC_POWER.resample('D').sum()
df2_AC = solar2GENdt.AC_POWER.resample('D').sum()
df2_DY = solar2GENdt.DAILY_YIELD.resample('D').last()
df2_TY = solar2GENdt.TOTAL_YIELD.resample('D').last()
df2_DC.tail()

In [ ]:
df = pd.merge( df2_DC, df2_AC, left_index=True, right_index=True) 
df_ = pd.merge( df2_DY ,df2_TY, left_index=True, right_index=True) 

df_2 = pd.merge( df ,df_, left_index=True, right_index=True) 

In [ ]:
#df_2.info()

### Comparison of daily produced currents and daily yields
I want to check if the generated energy will reflect in the daily and total yield values.  
Plant 1 is the only reference we have at disposal. 

In [ ]:
fig, ax= plt.subplots(2,2, figsize=(16, 10), sharex=True, sharey=True)
sns.pointplot(df_1.index, data=df_1, y=df_1.AC_POWER/85,label="AC POWER 1", join=False, ci="sd", ax=ax[0,0], ); #units="AC POWER 1"
sns.pointplot( df_1.index, data=df_1, y="DAILY_YIELD",label="DAILY YIELD 1", join=False, ci="sd", ax=ax[0,1]); plt.xticks([])
sns.pointplot(df_1.index, data=df_2, y=df_2.AC_POWER/85,label="AC POWER 2", join=False, ci="sd", ax=ax[1,0]); plt.xticks([])
sns.pointplot( df_1.index, data=df_2, y="DAILY_YIELD", label="DAILY_YIELD 2",join=False, ci="sd", ax=ax[1,1]);  # 
plt.xticks([]);

When I bring these values to the same scale, it is obvious that the generated energy is faulty picked up by the inverter. The daily yields for plant 2 are lower than what we could expect from its generated ac currents.  
Does the circuit lose energy somewhere? Maybe something else is wrong? Hardware or software?

### Day 2020-05-23
we focus on what happens on a "warmer" day

In [ ]:
dta= pd.to_datetime("2020-05-23 05:30:00"); dtb= pd.to_datetime("2020-05-23 19:00:00")
solar1GEN23MAY= solar1GEN[solar1GEN.DATE_TIME>= dta] #"2020-5-23 05:30:00"
solar1GEN23MAY= solar1GEN23MAY[solar1GEN23MAY.DATE_TIME <dtb]; #"2020-5-23 19:30:00"

In [ ]:
plt.figure(figsize=(16, 9))
ax= sns.lineplot(x=solar1GEN23MAY.DATE_TIME, y="DC_POWER", data=solar1GEN23MAY,color="forestgreen"); # kind="marker"
sns.lineplot(x=solar1GEN23MAY.DATE_TIME, y="AC_POWER", data=solar1GEN23MAY,color="indigo", ax=ax); 
plt.title('E generated (23 MAY)')
plt.grid("True"); 
plt.ylim=(250,1400); #plt.show()

The Inverters cannot keep up the conversion of energy to ac current when a certain amount of DC current - or amount of energy - has been generated. Inhibitors can be inner Inverter temperature, which is a function of the ambient temperature, the device material properties, sufficient ventilation and sufficient cover against direct sunlight.

### Chronological Heatmap of solar plant 1, May 23rd

In [ ]:
dta= pd.to_datetime("2020-05-23 05:30:00"); dtb= pd.to_datetime("2020-05-23 19:00:00")
solar1GEN_mei23_= solar1GEN[solar1GEN.DATE_TIME>= dta] # 2nd version !
solar1GEN_mei23_= solar1GEN_mei23_[solar1GEN_mei23_.DATE_TIME <dtb]; #"2020-5-23 19:30:00"

In [ ]:
solar1GEN_mei23_piv= solar1GEN_mei23_.pivot(index="SOURCE_KEY" , columns="DATE_TIME", values= "AC_POWER")

In [ ]:
solar1GEN_mei23_piv2= solar1GEN_mei23_.pivot(index="DATE_TIME", columns="SOURCE_KEY", values= "AC_POWER")
solar1GEN_mei23_piv2.head()

In [ ]:
plt.figure(figsize=(18, 15))
sns.heatmap( data=solar1GEN_mei23_piv);

## Inverter heatmap of plant 1

In [ ]:
#solar1GEN.head()

In [ ]:
solar1GEN_piv= solar1GEN[(solar1GEN.TIME>=  pd.to_datetime("06:00:00").time()) & (solar1GEN.TIME<=  pd.to_datetime("22:00:00").time())]
solar1GEN_piv= solar1GEN_piv.pivot_table( index="DATE_TIME", columns="SOURCE_KEY", values= "AC_POWER")#.max(
solar1GEN_piv

In [ ]:
solar1GEN_piv= solar1GEN_piv.dropna()
#solar1GEN_piv['TIME'] = solar1GEN_piv.index.apply(lambda x:x.time()) # ['DATE_TIME']
#solar1GEN_piv = solar1GEN_piv[(solar1GEN_piv.TIME >  pd.to_datetime("06:00:00")) & (solar1GEN_piv.TIME <=  pd.to_datetime("22:00:00"))]

In [ ]:
plt.figure(figsize=(18, 14))
sns.heatmap(data=solar1GEN_piv, ); #linewidths=0.1,  

Circuits A1 and A14 are always in service, it seems, while the rest is split up in 2 groups (8 and 12). These groups appear to work, or fall out, in turn. 
Take a look at these inverters: A5,A11, A12,A14,A16, A18,A19,A21, A22,A25.  

It is known that solar panel efficiency is  related to its placement, angle, and the sun's position. And high temperatures are known to reduce efficiency.  
As I noticed the solar panel inverter temperatures can reach 65°C, I suspect that overheated ones reduce or stop producing when a certain threshold is reached.  
We can spot 10 days where group fall out occurs: 2020-05-15,2020-05-19,2020-05-21,2020-05-22,2020-05-27, 2020-05-29,2020-06-02, 2020-06-04, 2020-06-05, 2020-06-06. We'll use this selection also for plant 2. 

In [ ]:
groupfallout= ["2020-05-15","2020-05-19","2020-05-21","2020-05-22","2020-05-27", "2020-05-29", 
               "2020-06-02", "2020-06-04", "2020-06-05", "2020-06-06"]
dayends=["2020-05-15 23:00:00","2020-05-19 23:00:00","2020-05-21 23:00:00","2020-05-22 23:00:00","2020-05-27 23:00:00","2020-05-29 23:00:00",
         "2020-06-02 23:00:00", "2020-06-04 23:00:00","2020-06-05 23:00:00","2020-06-06 23:00:00"]
dfx=pd.DataFrame()

df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-05-15 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-05-15 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-05-19 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-05-19 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-05-21 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-05-21 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-05-22 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-05-23 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-05-27 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-05-27 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-05-29 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-05-29 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-06-02 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-06-02 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-06-04 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-06-04 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-06-05 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-06-05 23:00:00"))]
dfx=dfx.append(df)
df = solar1GEN[(solar1GEN.DATE_TIME >  pd.to_datetime("2020-06-06 06:00:00")) & (solar1GEN.DATE_TIME <=  pd.to_datetime("2020-06-06 23:00:00"))]
dfx=dfx.append(df)

In [ ]:
outgroup=["A5","A11", "A12","A14","A16", "A18","A19","A21", "A22","A25"]
solar1GEN_out= dfx.loc[dfx.SOURCE_KEY.isin(outgroup) ]
solar1GEN_out= solar1GEN_out.set_index("DATE_TIME")
solar1GEN_out_merg = solar1Sensordt.merge(solar1GEN_out, left_index=True, right_index=True ) # solar1GEN_A12

### Filter out low production times for plant 1 ...

In [ ]:
def Rollingsum(df,t):
    df["AMB_rol"+str(t)] =df.AMBIENT_TEMPERATURE.rolling(t).sum()
    df["MOD_rol"+str(t)] =df.MODULE_TEMPERATURE.rolling(t).sum()
    df["D_Y_dif"+str(t)] =df.DAILY_YIELD.diff(1)  #rolling(t)
    df["D_Y_rol"+str(t)] =df["D_Y_dif"+str(t)].rolling(t).sum()
    df["AC_rol"+str(t)] =df.AC_POWER.rolling(t).sum()
    df["DC_rol"+str(t)] =df.DC_POWER.rolling(t).sum()

In [ ]:
Rollingsum(solar1GEN_out_merg,4)

In [ ]:
#solar1GEN_out_merg.sample(6)

In [ ]:
solar1GEN_out_merghigh =solar1GEN_out_merg.loc[solar1GEN_out_merg.MODULE_TEMPERATURE> 30] 
solar1GEN_out_merghigh.head()

## Panel temperature and ambient temperature
a plot of a "normal" functioning solar panel plant

In [ ]:
plt.figure(figsize=(20, 10))
sns.lineplot(x=solar1GEN_out_merghigh.index, y="MODULE_TEMPERATURE", data=solar1GEN_out_merghigh, lw=0.85); # "DATE_TIME"
sns.lineplot(x=solar1GEN_out_merghigh.index, y="AMBIENT_TEMPERATURE", data=solar1GEN_out_merghigh, lw=0.95); 
plt.xlim( pd.to_datetime("2020-05-29"), pd.to_datetime("2020-06-06")); 
plt.ylim=(25,65); 

In [ ]:
sns.scatterplot(x="MODULE_TEMPERATURE" ,y='AMBIENT_TEMPERATURE', data=solar1GEN_out_merghigh, hue="AC_POWER",s=7); 

Note the many dark points for high current above 55°C. 

In [ ]:
sns.scatterplot(x="MODULE_TEMPERATURE" ,y='AMBIENT_TEMPERATURE', data=solar1GEN_out_merghigh, hue="DAILY_YIELD",s=8);

###  Filter out low production times for plant 2
..., but we keep all inverters 
Produced current and yield on 2020-06-04

In [ ]:
dfx=pd.DataFrame()

df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-05-15 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-05-15 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-05-19 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-05-19 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-05-21 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-05-21 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-05-22 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-05-23 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-05-27 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-05-27 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-05-29 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-05-29 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-06-02 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-06-02 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-06-04 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-06-04 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-06-05 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-06-05 23:00:00"))]
dfx=dfx.append(df)
df = solar2GEN[(solar2GEN.DATE_TIME >  pd.to_datetime("2020-06-06 06:00:00")) & (solar2GEN.DATE_TIME <=  pd.to_datetime("2020-06-06 23:00:00"))]
dfx=dfx.append(df)

In [ ]:
solar2GEN_out= dfx#.loc[dfx.SOURCE_KEY.isin(outgroup) ]
solar2GEN_out= solar2GEN_out.set_index("DATE_TIME")
solar2GEN_out_merg = solar1Sensordt.merge(solar2GEN_out, left_index=True, right_index=True ) # solar2GEN_A12

In [ ]:
solar2GEN_out_merghigh =solar2GEN_out_merg.loc[solar2GEN_out_merg.MODULE_TEMPERATURE> 30] 

In [ ]:
plt.figure(figsize=(20, 12))
sns.lineplot(x=solar2GEN_out_merghigh.index , y="MODULE_TEMPERATURE", data=solar2GEN_out_merghigh, lw=0.85); 
sns.lineplot(x=solar2GEN_out_merghigh.index , y="AMBIENT_TEMPERATURE", data=solar2GEN_out_merghigh, lw=0.95); 
plt.xlim(pd.to_datetime("2020-05-21") ,pd.to_datetime("2020-05-30")); plt.ylim=(30,63); 

In [ ]:
sns.scatterplot(x="MODULE_TEMPERATURE" ,y='AMBIENT_TEMPERATURE', data=solar2GEN_out_merghigh, hue="AC_POWER",s=9); 

Compared to plant 1, there are no so many dark spots above 55° for plant 2. Daily yields are even more depressed. 

rol4 =solar1GEN_out_merg[["AMB_rol4","MOD_rol4","D_Y_rol4","AC_rol4", "DC_rol4"]] # "DATE_TIME",
rol4= rol4[(rol4.index >pd.to_datetime("2020-06-04 08:00:00")) & (rol4.index <pd.to_datetime("2020-06-05 20:00:00"))]

fig, ax= plt.subplots(1,1, figsize=(20, 10))
sns.lineplot(x=rol4.index, y=rol4.MOD_rol4, data=rol4,label="MOD_rol4", lw=0.95); # "DATE_TIME", 
sns.lineplot(x=rol4.index, y=rol4.D_Y_rol4*2, data=rol4,label="D_Y_rol4 *2", lw=0.95); 
sns.lineplot(x=rol4.index, y= rol4.DC_rol4/180, data=rol4,label="DC_rol4 /180", lw=0.95); 
ax.set_xlim(pd.to_datetime("2020-06-04 08:00:00"),pd.to_datetime("2020-06-05 20:00:00")); 
ax.set_ylim(75,275);  

### Rolling sum and diff


In [ ]:
def Rollingsum(df,t):
    df["AMB_rol"+str(t)] =df.AMBIENT_TEMPERATURE.rolling(t).sum()
    df["MOD_rol"+str(t)] =df.MODULE_TEMPERATURE.rolling(t).sum()
    df["D_Y_dif"+str(t)] =df.DAILY_YIELD.diff(1)  #rolling(t)
    df["D_Y_rol"+str(t)] =df["D_Y_dif"+str(t)].rolling(t).sum()
    df["AC_rol"+str(t)] =df.AC_POWER.rolling(t).sum()
    df["DC_rol"+str(t)] =df.DC_POWER.rolling(t).sum()

In [ ]:
Rollingsum(solar1GEN_out_merg,4)

In [ ]:
solar1GEN_out_merg.sample(1)

### Produced current and yield on plant 1, 2020-06-04

In [ ]:
rol4 =solar1GEN_out_merg[["AMB_rol4","MOD_rol4","D_Y_rol4","AC_rol4", "DC_rol4"]] # "DATE_TIME",
rol4= rol4[(rol4.index >"2020-06-04 05:00:00") & (rol4.index <"2020-06-05")]
rol4= rol4[rol4.D_Y_rol4 > -2000]

plt.style.use("ggplot")
fig, ax= plt.subplots(1,1, figsize=(16, 10))
sns.lineplot(x=rol4.index, y=rol4.MOD_rol4, data=rol4, label="MOD_rol4", lw=0.95); 
sns.lineplot(x=rol4.index , y=rol4.D_Y_rol4*2, data=rol4, label="D_Y_rol4 $*2$", lw=0.95); 
sns.lineplot(x=rol4.index, y= rol4.DC_rol4/18, data=rol4, label="DC_rol4 $/18$", lw=0.95); 
plt.axhline(y=180, xmin=0, xmax=1, ls=":", color="violet") # 45°C
#plt.gca()
dt04 =pd.to_datetime("2020-06-04 05:00:00"); dt05 =pd.to_datetime("2020-06-04 21:00:00"); 
ax.set_xlim( dt04, dt05) #plt.xlim( dt04, dt05)
ax.set_ylim(75,275)
plt.grid(True)
plt.title("Produced current and yield for plant 1, 2020-06-04"); 

From 10 AM onwards, the yield tries to follow the (amount of produced) DC current, but above 210 =52.5 °C, the difference in height btw. DC and yield widens. 
The yield curve stays behind the DC curve. From this point on, the DC - AC conversion is no longer optimal. The inverters seem to normalize again after inner temperature falling below 50°C. 

### Produced current and yield for plant2, 2020-06-04

In [ ]:
Rollingsum(solar2GEN_out_merg,4)

rol4 =solar2GEN_out_merg[["AMB_rol4","MOD_rol4","D_Y_rol4","AC_rol4", "DC_rol4"]] # "DATE_TIME",
rol4= rol4[(rol4.index >="2020-06-04 05:00:00") & (rol4.index <"2020-06-05")]
rol4= rol4[rol4.D_Y_rol4 > -2000]

plt.style.use("ggplot")
fig, ax= plt.subplots(1,1, figsize=(16, 10))
sns.lineplot(x=rol4.index, y=rol4.MOD_rol4, data=rol4,label="MOD_rol4", lw=0.95);  #DATE_TIME
sns.lineplot(x=rol4.index, y=rol4.D_Y_rol4*4, data=rol4,label="D_Y_rol4 $*4$", lw=0.95); 
sns.lineplot(x=rol4.index, y= rol4.DC_rol4/18, data=rol4,label="DC_rol4 $/18$", lw=0.95); 
ax.set_xlim(pd.to_datetime("2020-06-04 06:00:00"),pd.to_datetime("2020-06-04 21:00:00"))
ax.set_ylim(50,250)
plt.axhline(y=180, xmin=0, xmax=1, ls=":") # 45°C
plt.title("Produced current and yield for plant2, 2020-06-04"); 

The daily yields (orange) have been multiplied by 2 in this plot, and currents are 10 times more than those values used in the previous plot of plant 1.  
The panel temperature values (blue) have not been changed, so they remain at 4 times their real value: 200 in the plot means 50°C in reality. 

---

## Interludium: introduction to photovoltaic system performance

### N.M. Pearsall, in The Performance of Photovoltaic (PV) System, 2017
#### **1.3.4 Other performance parameters**

The three performance parameters discussed in the previous section express the overall performance of the system but it is sometimes useful to consider the specific performance of certain parts of the system in order to ensure the correct design and operational choices.

It is possible to determine the inverter efficiency if measurements of both DC input and AC output are provided. In general, the efficiency of a PV inverter is a function of the input power and input voltage, with a typical set of efficiency curves being shown in Fig. 1.4. At medium to high light levels and therefore input power from the array, the inverter has a high efficiency, generally well in excess of 90%. At low irradiance levels, the efficiency drops off sharply. This means that we can determine an optimum inverter capacity in comparison with the array capacity, such that the balance between energy loss at the low irradiance end due to reducing efficiency is balanced against energy loss at the high irradiance end due to limiting because of the maximum inverter capacity. This is illustrated in Fig. 1.5, assuming that the basic shape of the inverter efficiency curve does not depend on inverter capacity. Clearly, the balance between energy generation at low and high irradiance values is dependent on the climate and, therefore, so is the optimum inverter/array ratio, with the general approach of a reduction in this ratio as the latitude increases.


*Fig. 1.4. Typical inverter efficiency curve as a function of DC input voltage. The rated input voltage will be a value around the middle of the allowed voltage range. Curves are also shown for higher and lower voltages, closer to the limits of the voltage range.*
<img src="./kaggle/input/inverter-efficiency/inverter efficiency1.jpg" embed="True">

*Fig. 1.5. Comparison of inverter efficiency for two inverters, A and B, where the rated capacity of B is 75% of that of A. The general shape of the efficiency curve is assumed to be unaffected by the capacity and the capacity of the PV array is assumed to be equal to A. It can be seen that inverter B provides more output than A at low irradiance values but limits at PV array output values higher than 75% of rated capacity. The optimum capacity of the inverter will depend on the ratio between the energy received at low irradiance and at high irradiance for that particular system.* <img src="kaggle/input/inverter-efficiency/inverter efficiency2.jpg">

Because of the variation of efficiency with input power, and therefore irradiance on the array, the average operating efficiency of the inverter will vary with climate. In general, the technical information for a PV inverter will include both the peak efficiency (usually between 95% and 98% depending on the inverter technology) and a weighted efficiency to account for the operation at different irradiance levels. In Europe, this weighted efficiency is termed the Euro efficiency and can be represented by Eq. (1.10), as follows: (1.10) 
$$Euro_{efficiency}= 0.03*Eff_{5%} +0.06* Eff_{10%} +0.13*Eff_{20%} +0.1*Eff_{30%} +0.48*Eff_{50%} +0.2*Eff_{100%}$$

Here, Eff_5% is the DC to AC conversion efficiency at an input power of 5% of the inverter capacity, with similar definitions for the other terms. The weighting reflects the amount of energy predicted to be gained at each energy level. An alternative weighting, using the same rationale and approach, is the California Energy Commission or CEC efficiency, calculated as follows [10]: (1.11) 
$$CEC_{efficiency}= 0.04*Eff_{10%} +0.05*Eff_{20%} + 0.12*Eff_{30%} +0.21*Eff_{50%} +0.53*Eff_{75%} +0.05*Eff_{100%}$$

Here the weighting factors are more heavily biased towards the higher irradiance levels, to reflect the difference between the climates in the southwestern USA and central Europe. The use of either the Euro or CEC efficiency gives a lower but more representative value than the peak efficiency in terms of considering the overall inverter efficiency across a period of operation. Clearly, other weightings could be developed for other climate options, although there needs to be consensus within the community for widespread use.

For a stand-alone PV system, the important parameter is not the total energy generated but whether the load is met for the required time, that is, the service provided by the system. This is sometimes assessed by direct reference to the load, for example, amount of water pumped, amount of product manufactured using PV electricity. However, it is also possible to define parameters to express the system performance, such as the total amount of time for which the load is not met (to be compared with the loss of load probability defined in the system design) and the battery index, which is the percentage of days in a given period when full charge of the batteries in the system is achieved. In general, values over 30% are considered as good, although very high values may indicate an oversized array.

---

## Temperature and irradiation sensor data
Higher ambient and inner temperatures reduce the capability of inverters to convert energy. 

In [ ]:
sensors1_list =solar1Sensor.SOURCE_KEY.unique()
solar1Sensor= solar1Sensor.drop(["SOURCE_KEY"], axis=1) # "PLANT_ID",
solar1Sensordt= solar1Sensor.set_index("DATE_TIME",  ) #inplace=True
solar1Sensordt.head()

In [ ]:
sensors2_list =solar2Sensor.SOURCE_KEY.unique()
solar2Sensor= solar2Sensor.drop(["SOURCE_KEY"], axis=1) # "PLANT_ID",
solar2Sensordt= solar2Sensor.set_index("DATE_TIME",  ) #inplace=True

In [ ]:
fig, ax=  plt.subplots(1,1, figsize=(18, 7.6))  
plt.gca()  
sns.lineplot( data=solar1Sensordt);  

## Module temperatures tempers the amount of generated current

In [ ]:
dta= pd.to_datetime("2020-05-23 05:30:00"); dtb= pd.to_datetime("2020-05-23 19:30:00")
Sensor1dt_mei23_= solar1Sensordt[solar1Sensordt.index>= dta] #"2020-5-23 05:30:00"
Sensor1dt_mei23_= Sensor1dt_mei23_[Sensor1dt_mei23_.index <=dtb]; #

In [ ]:
dta= pd.to_datetime("2020-05-01 05:30:00"); dtb= pd.to_datetime("2020-05-31 19:30:00")
Sensor2dt_mei= solar2Sensordt[solar2Sensordt.index>= dta] #"2020-5-23 05:30:00"
Sensor2dt_mei= Sensor2dt_mei[Sensor2dt_mei.index <=dtb]; #

In [ ]:
mei23_merg =solar1GEN_mei23_.merge( solar1Sensordt, right_index=True, left_on="DATE_TIME")

In [ ]:
mei23B_merg = Sensor2dt_mei23_.merge(solar2GEN,left_index=True, right_on="DATE_TIME" )

mei_B_merg = Sensor2dt_mei.merge(solar2GENdt,left_index=True, right_index=True )

In [ ]:
mei_B_merg.info()

In [ ]:
mei_B_merg= mei_B_merg.set_index("DATE_TIME")

In [ ]:
mei_B_merg.head()

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(18, 7.6))
sns.lineplot(x=mei23_merg.DATE_TIME,y=mei23_merg.DC_POWER/850, data=mei23_merg,label="DC $/85$"); 
#sns.lineplot(x=mei23_merg.DATE_TIME,y=mei23_merg.AC_POWER/85, data=mei23_merg,label="AC $/85$");
sns.lineplot(x=mei23_merg.DATE_TIME,y=mei23_merg.IRRADIATION*30, data=mei23_merg, label="Irradiation $*30$"); 
sns.lineplot(x=mei23_merg.DATE_TIME,y=mei23_merg.MODULE_TEMPERATURE, data=mei23_merg,label="Module temp."); 

>It appears that the current stalls and slows down at 63°C, and it resumes an upward trend only after the inverter temperature falls below 58°C. 

##  Correlation btw.  ac current, ambient temperature and module temperature.  
Comparison of the standard deviations of some measurements by a rolling window of 3. 

In [ ]:
mei_B4_merg= mei_B_merg[mei_B_merg.SOURCE_KEY=="B4"] # was renamed
dta= pd.to_datetime("2020-05-23 05:00:00"); dtb= pd.to_datetime("2020-05-23 20:30:00")
mei_B4_merg= mei_B4_merg[mei_B4_merg.index>= dta] #"2020-5-23 05:30:00"pd.to_datetime(
mei_B4_merg= mei_B4_merg[mei_B4_merg.index <=dtb]; #pd.to_datetime(

mei_B4_merg["AC_std"]= mei_B4_merg.AC_POWER.rolling(3).std(min_periods=1); 
mei_B4_merg["AC_avg"]= mei_B4_merg.AC_POWER.rolling(3).mean(); 
mei_B4_merg["RAD_std"]= mei_B4_merg.IRRADIATION.rolling(3).std(); 
mei_B4_merg["Mod_T_std"]= mei_B4_merg.MODULE_TEMPERATURE.rolling(3).std(); 
mei_B4_merg["Amb_T_std"]= mei_B4_merg.AMBIENT_TEMPERATURE.rolling(3).std(min_periods=1);

A badly behaving inverter: B4

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(17, 7)) #DATE_TIME
sns.lineplot(x=mei_B4_merg.index ,y=mei_B4_merg.Amb_T_std, data=mei_B4_merg,label="Ambient temperature std"); 
sns.lineplot(x=mei_B4_merg.index ,y=mei_B4_merg.AC_avg/1200,data=mei_B4_merg,label="AC avg");
sns.lineplot(x=mei_B4_merg.index ,y=mei_B4_merg.AC_std/180,  data=mei_B4_merg,label="AC std");
sns.lineplot(x=mei_B4_merg.index ,y=mei_B4_merg.RAD_std*5,   data=mei_B4_merg, label="Radiation std"); #
sns.lineplot(x=mei_B4_merg.index ,y=mei_B4_merg.Mod_T_std/5, data=mei_B4_merg,label="Module temp. std"); #
plt.title("Inv. B4: standard deviations of current and temperatures"); 
ax.set_ylim(0,1.2) #plt.ylim(0,1.2); 


### Conclusion:

Inverter B4 doesn't slow down the current in its circuit by regulating the voltage. The current drop starts at 10:00 - 10:30.
I noticed that on average the temperature here is 4° higher than at plant 1. What happens here is that the voltage drops happen sooner (at a lower V.) with every degree of panel temperature increase. 

   1. It could be the case that this inverter was not configured for optimal functioning, in hot days, of this location.
   2. Maybe the blocking and by-pass diodes between the panels are not in place, or broken.
   3. Chose better material for PV cells, like pSi, aSi, InGa.

